In [ ]:
%matplotlib inline

import numpy as np

from sklearn_theano.feature_extraction import fetch_overfeat_weights_and_biases

import menpo.io as mio
from menpo.visualize import visualize_images

from alaborticcv2015.deepconvkernel.pretrained import PreTrainedLDCN
from alaborticcv2015.utils import greyscale, normalize_norm

# OverFeat LDCN 

Choose whether to work with color or greyscale images:

In [ ]:
color = False

Obtain OverFeat filters:

In [ ]:
weights, _ = fetch_overfeat_weights_and_biases(large_network=True)

In [ ]:
filters = [np.asarray(w) for w in weights]

if not color:
    grey_filters = np.empty((filters[0].shape[0], 1) + filters[0].shape[-2:])
    for j, f in enumerate(filters[0]):
        grey_filters[j] = greyscale(f, mode='average')
    filters[0] = grey_filters

Number and shape of the filters per layer:

In [ ]:
for j, w in enumerate(filters):
    print 'Layer:', j,  '\t # of filters:', w.shape[0], '\t shape:', w.shape[1:]

In [ ]:
filters[0] = filters[0]
filters[1] = filters[1][:5]

Build a Pre-Trained LinDeepConvNet with the filters of the first 2 layers:

In [ ]:
net = PreTrainedLDCN(norm_func=normalize_norm)
net.build_network(filters[:2])

In [ ]:
print '# of layers:', net.n_layers
print '# of filters:', net.n_filters
print '# of filters per layer:', net.n_filters_layer
print 'filters shape:', net.filters_shape

Visualize the previous filters using the previous LinDeepConvNet:

In [ ]:
filters = net.filters_spatial()

First layer spatial filters:

In [ ]:
visualize_images(filters[0])

Second layer spatial filters:

In [ ]:
visualize_images(filters[1])

We can also visualize the previous filters on the frequency domain:

In [ ]:
filters_frequecy = net.filters_frequency(ext_shape=(50, 50))

First layer filters in the frequency domain:

In [ ]:
visualize_images(filters_frequecy[0])

Second layer filters in the frequency domain:

In [ ]:
visualize_images(filters_frequecy[1])

Visualize Deep Convolutional Kernels:

In [ ]:
kernels = net.kernels_spatial(ext_shape=(50, 50))

In [ ]:
visualize_images(kernels)

In [ ]:
kernels = net.kernels_frequency(ext_shape=(50, 50))

In [ ]:
visualize_images(kernels)

Load images:

In [ ]:
images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/trainset/', verbose=True, 
                           max_images=10):
    i.crop_to_landmarks_proportion_inplace(1)
    i = i.rescale_landmarks_to_diagonal_range(100)
    if not color:
        if i.n_channels == 3:
            i = i.as_greyscale(mode='average')
    images.append(i)

Choose one of the previous images:

In [ ]:
image = images[1]

Compute network features:

In [ ]:
net.network_response(image).view_widget()

Compute deep convolutional kernel features:

In [ ]:
net.kernel_response(image).view_widget()

Check Network and Kernel equivalence:

In [ ]:
network_r = net.network_response(image)
kernel_r = net.kernel_response(image)

print network_r.as_vector().dot(network_r.as_vector())
print kernel_r.as_vector().dot(kernel_r.as_vector())